In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.pipeline import Pipeline, CustomFilter
from quantopian.pipeline.factors import CustomFactor
from quantopian.pipeline.data import Fundamentals
#import quantopian.pipeline.data.morningstar import Fundamentals
from quantopian.pipeline.data.builtin import USEquityPricing
import numpy as np
import pandas as pd

In [ ]:
class Ebitda_to_EV(CustomFactor):
    '''
    The book uses EBITDA/EV. Fundamental data gives us access to EV/EBITDA. 
    EBITDA/EV = 1/(EV/EBITDA)
    If result gives us an infinite number, that is changed to NaN. This will 
    be handled when passing it to the Quantiles function
    '''
    window_length = 1
    inputs=[Fundamentals.ev_to_ebitda]
    
    def compute(self, today, assets, out, ev_ebitda):
        result = 1 / ev_ebitda[-1]
        result[np.isinf(result)] = np.nan
        out[:] = result

In [ ]:
def make_pipeline():
    # All Stocks universe consist of stocks with a market capitalization in excess of $200 mil
    all_stock_universe = Fundamentals.market_cap.latest > 200000000

    ebitda_ev = Ebitda_to_EV()

    ebitda_ev_zscore = ebitda_ev.zscore()

    return Pipeline(
        columns={
            'factor' : ebitda_ev_zscore,
        },
        screen = all_stock_universe & ebitda_ev.notnull()
    )

In [ ]:
# Import run_pipeline method
from quantopian.research import run_pipeline

# Specify a time range to evaluate
period_start = '2005-01-01'
period_end = '2018-01-01'

# Execute pipeline over evaluation period
pipeline_output = run_pipeline(
    make_pipeline(),
    start_date=period_start,
    end_date=period_end
)

print "There are %d assets in this universe." % len(pipeline_output.index.levels[1])

In [ ]:
import alphalens

#Gets each unique stock ticker and puts it in assets
assets = pipeline_output.index.levels[1]

#gets pricing data. Needs a month before and after. Dunno why.
pricing = get_pricing(assets, start_date='2005-01-01', end_date='2018-01-01', fields='open_price')

In [ ]:
# Ingest and format data
#factor = pipeline_output['factor']
factor_data = alphalens.utils.get_clean_factor_and_forward_returns(pipeline_output,
                                                                   pricing,
                                                                   periods = (10, 20, 30),
                                                                  quantiles = 10)

In [ ]:
alphalens.tears.create_information_tear_sheet(factor_data)

In [ ]:
alphalens.performance.factor_alpha_beta(factor_data)

In [ ]:
alphalens.tears.create_summary_tear_sheet(factor_data)